In [ ]:
#Though it works as expected, its loss makes it unviable for forecasting
#usage, trying to optimize...
#TODO 

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import pandas as pd
from predict import date_to_int, train_test_split, create_sequences
from sklearn.preprocessing import MinMaxScaler

TRAIN_RATIO = 0.8
TIMESTEP = 12


In [ ]:
df = pd.read_csv('data/yf_monthly/AAPL.csv')
df['Date'] = [date_to_int(date) for date in df['Date']]

X_sc = MinMaxScaler()
scaled_X = X_sc.fit_transform(df)

y_sc = MinMaxScaler()
scaled_y = y_sc.fit_transform(df[['Close']])

In [ ]:
X, y = create_sequences(scaled_X, scaled_y, TIMESTEP)
X_train, y_train, X_test, y_test = train_test_split(X, y, TRAIN_RATIO)

In [ ]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(LSTM(units=50, input_shape=input_shape, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')
        
    return model

In [ ]:
model = create_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

In [ ]:
#Testing purposes
last_sequence = X_test[-1].reshape(1, X_test[-1].shape[0], X_test[-1].shape[1])

scaled_prediction = model.predict(last_sequence)

prediction = y_sc.inverse_transform(scaled_prediction)

last_y = y_sc.inverse_transform([y_test[-1]])
print(last_y, prediction)


